In [1]:
import os                                                 # to set current working directory 
import numpy as np                                        # arrays and matrix math
import pandas as pd                                       # DataFrames
import matplotlib.pyplot as plt                           # plotting
cmap = plt.cm.inferno                                     # color map
import geostatspy.geostats as geostats
import geostatspy.GSLIB as GSLIB
import math # For n_effective calculations
import scipy # For n_effective calculations
import time
import json

from statistics import mean
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from IPython.display import clear_output

%matplotlib inline

In [12]:
with open('.workflow_ignore.txt') as f:
    ignore_list = f.read().splitlines()

filename_list = [f.path.split('/')[-1] for f in os.scandir('dataset/') if(f.is_dir() and f.path.split('/')[-1] not in ignore_list)] # Getting files

for filenum, filename in enumerate(filename_list):
    #### READ DATA AND PARSE
    df = pd.read_csv('dataset/' + filename + '/' + filename + '.csv')
    
    # We are sampling in every 500 meter for a realistic scenario
    # The data where every 100x100 isotropic cell is stored is going
    # to be our truth model for the results.
    df = df.iloc[::5] 

    predictors = df.iloc[:,2:-1].columns.values
    response = df.columns[-1]
    predictors_and_response = df.iloc[:,2:].columns.values

    num_predictors = predictors.shape[0] # NO NEED, but defined just in case
    num_response = 1 # NO NEED, but defined just in case
    num_predictors_and_response = predictors_and_response.shape[0] # Used for loops

    df_train, df_test = train_test_split(df, test_size=0.5, random_state=73073)
    X_train = df_train.loc[:, predictors].values
    y_train = df_train.loc[:, response].values
    X_test = df_test.loc[:, predictors].values
    y_test = df_test.loc[:, response].values
    #### READ REQUIRED PARAMS
    with open('dataset/' + filename + '/' + filename + '_var_params_and_n_eff.json', 'r') as f:
        data = json.load(f)

    #### ASSIGN VARIABLES FROM PARAMS
    n_eff_calculated = data['n_eff_calculated']
    n_standard = y_train.shape[0]

    #### HYPERPARAMETER TUNING:

    L = 20
    max_num_leaf_nodes = [i for i in range(2,n_standard//3,20)]
    vals = list(range(y_train.shape[0])) # We will draw from the indexes and then we will slice the dataset with these indexes. We do this because data is multivariate
    n_eff_list = sorted(set([n_eff_calculated] + [i for i in range(10, df_train.shape[0]+1, 10)] + [df_train.shape[0]]))
    n_eff_params = {}
    n_eff_train_params = {}
    hyperparameter_error_spatial = []
    hyperparameter_error_standard = []

    hyperparameter_mse_test_dict = {}
    hyperparameter_mse_train_dict = {}

    for n_eff in n_eff_list:
        hyperparameter_mse_test_dict[n_eff] = {}
        hyperparameter_mse_train_dict[n_eff] = {}
        mse_init = 9999
        mse_train_init = 9999
        node_init = 0
        node_train_init = 0
        for node in max_num_leaf_nodes:
            regressor = DecisionTreeRegressor(max_leaf_nodes = node,random_state = 73073) # One decision tree
            y_pred = np.zeros(y_test.shape)
            y_pred_train = np.zeros(y_train.shape)

            for i in range(L):
                instance = np.random.choice(vals, size=n_eff, replace=True) # Size is equal to n_effective
                # Fitting standard bootstrap and storing the predictions for one realization
                dt = regressor.fit(X_train[instance], y_train[instance])
                y_pred = y_pred + dt.predict(X_test)
                y_pred_train = y_pred_train + dt.predict(X_train)

            y_pred /= (L+1)
            y_pred_train /= (L+1)
            mse = mean_squared_error(y_test, y_pred)
            mse_train = mean_squared_error(y_train, y_pred_train)
            if(mse<mse_init):
                mse_init = mse
                node_init = node
            hyperparameter_mse_test_dict[n_eff][node] = mse

            if(mse_train<mse_train_init):
                mse_train_init = mse_train
                node_train_init = node
            hyperparameter_mse_train_dict[n_eff][node] = mse_train

        n_eff_params[n_eff] = node_init
        n_eff_train_params[n_eff] = node_train_init

    # Obtain hyperparameter tuning with training data
    json_string = json.dumps(hyperparameter_mse_train_dict)
    with open('dataset/' + filename + '/' + filename + '_hyperparameter_mse_train_dict.json', 'w') as f:
        f.write(json_string)

    # Obtain hyperparameter tuning with testing data
    json_string = json.dumps(hyperparameter_mse_test_dict)
    with open('dataset/' + filename + '/' + filename + '_hyperparameter_mse_test_dict.json', 'w') as f:
        f.write(json_string)

    # Obtain tuned hyperparameter for each n
    json_string = json.dumps(n_eff_params)
    with open('dataset/' + filename + '/' + filename + '_hyperparameter_for_each_n.json', 'w') as f:
        f.write(json_string)

    vals = list(range(y_train.shape[0])) # We will draw from the indexes and then we will slice the dataset with these indexes. We do this because data is multivariate
    len_vals = len(vals) # Total size of train data for easiness of syntax
    L = 100 # Number of realizations for each n_effective
    L_2 = 10 # Bootstrap of Bootstrap number
    n_bins = 50 # Number of bins for histograms

    n_eff_broadcast = np.array([n_eff_list for i in range(L_2)])

    # Creating empty lists to store bootstrapped samples

    boots_arr = [[[None for i in range(L)] for j in range(len(n_eff_list))] for k in range(L_2)]

    # Creating empty lists to store Decision Tree regression results

    reals = [[[None for i in range(L)] for j in range(len(n_eff_list))] for k in range(L_2)]

    # Creating empty lists to store Decision Tree regression Training Fits

    trains = [[[None for i in range(L)] for j in range(len(n_eff_list))] for k in range(L_2)]

    # Creating empty lists to store the aggregate (bagged) of L realizations for each n_effective

    bags = [[None for i in range(len(n_eff_list))] for j in range(L_2)]

    # Creating empty lists to store the aggregate (bagged) of L realizations for each n_effective (TRAINING DATA)

    bags_trains = [[None for i in range(len(n_eff_list))] for j in range(L_2)]

    # Creating empty lists to store MSE for each bagged result.

    mse_test = [[None for i in range(len(n_eff_list))] for j in range(L_2)]

    # Creating empty lists to store MSE for each bagged result. (TRAINING DATA)

    mse_train = [[None for i in range(len(n_eff_list))] for j in range(L_2)]

    # Main Loop

    # For each n_effective (number of draws or n_draw here) in n_effective list
    for n_idx, n_draw in enumerate(n_eff_list):
        node = n_eff_params[n_draw]
        regressor = DecisionTreeRegressor(max_leaf_nodes=node, random_state = 73073) # One decision tree
        for j in range(L_2):
            # We will have L (100) realizations. For each of them, do the following:
            for i in range(L):
                # Sample with replacement (bootstrap). Note that we sample INDEXES, not the values. Then, using these indexes, we will draw from X_train
                spt_idxs = np.random.choice(vals, size=n_draw, replace=True) # Size is equal to n_effective

                #Storing the Indexes as bootstrap realizations so that we can access them later if we need
                boots_arr[j][n_idx][i] = spt_idxs

                # Fitting spatial bootstrap and storing the predictions for one realization
                dt = regressor.fit(X_train[spt_idxs], y_train[spt_idxs])
                trains[j][n_idx][i] = dt.predict(X_train)
                reals[j][n_idx][i] = dt.predict(X_test)

            # After L realizations made, we aggregate the predictions (bagging) and store them in respective lists    
            bags[j][n_idx] = np.mean(reals[j][n_idx], axis=0)
            bags_trains[j][n_idx] = np.mean(trains[j][n_idx], axis=0)

            # Last, we calculate MSE for each aggregation.
            mse_test[j][n_idx] = mean_squared_error(y_test, bags[j][n_idx])
            mse_train[j][n_idx] = mean_squared_error(y_train, bags_trains[j][n_idx])


    boots_arr = np.asarray(boots_arr, dtype=object)
    reals = np.asarray(reals)
    bags = np.asarray(bags)
    bags_trains = np.asarray(bags_trains)
    mse_test = np.asarray(mse_test)
    mse_train = np.asarray(mse_train)

    mse_train_dict = {}
    mse_test_dict = {}

    for idx in range(len(n_eff_list)):
        mse_train_dict.update({n_eff_list[idx]: mse_train[:,idx].tolist()})
        mse_test_dict.update({n_eff_list[idx]: mse_test[:,idx].tolist()})

    # Obtain mse for training
    json_string = json.dumps(mse_train_dict)
    with open('dataset/' + filename + '/' + filename + '_mse_train.json', 'w') as f:
        f.write(json_string)

    # Obtain mse for training
    json_string = json.dumps(mse_test_dict)
    with open('dataset/' + filename + '/' + filename + '_mse_test.json', 'w') as f:
        f.write(json_string)

    mse_ratio_dict = {filename + '/' + filename: mean(mse_test_dict[n_standard]) / mean(mse_test_dict[n_eff_calculated])}

    # Obtain mse ratios
    json_string = json.dumps(mse_ratio_dict)
    with open('dataset/' + filename + '/' + filename + '_mse_ratio.json', 'w') as f:
        f.write(json_string)
    print(str(filenum+1) + '/' + str(len(filename_list)))
    clear_output(wait=True)
    
    break

1/4
